# Initialisation

In [1]:
### Load modules and data

import numpy as np
import itertools as it
import matplotlib.pyplot as plt
import math
from scipy import stats
import pdb
from sklearn import preprocessing
import copy
from unittest import *
import itertools
from bidict import bidict

%matplotlib inline


## Trait simulation

### Explanation

Given genotype data and an LD structure, simulate a trait which is linearly associated with a variant, or a set of variants. Here I generate a large $m \times n$ matrix ($m$=number of samples, $n$=number of SNPs), with $0,1,2$ as elements.

Then, I can choose a set of SNPs, and from these SNPs I generate a trait with a linear model. In this 

Following this, I try to recover these sets of SNPs. I generate p-values for each SNP being associated with the trait, by individually building univariate linear models for each SNPs, as I understand summary statistics are generated.

### Implementation

In [2]:
### Sample genotypes

def simulate_genotype(n,m,geno_dist):
    """
    Simulate a genotype of n samples and m causal SNPs with specified genotype distribution for (0,1,2).
    """
    X=np.zeros([n,m])
    for i in range(m):
        X[:,i] = [np.random.choice(a=[0,1,2],p=geno_dist) for x in range(n)]
    return np.array(X)

###example
# X = simulate_genotype(n=10000,m=30,geno_dist=[0.85,0.1,0.05])

def simulate_traits(X,snp_ratios,beta_var):
    """
    X is genotype information.
    
    beta_var is the total heteritability and must be between 0 and 1.
    
    snp_ratio is a dictionary where dictionary values signify the ratio of how much each snp explains 
    of the total heritability.
    
    For example beta_var = 0.6, snp_ratios = {3:1, 5:1, 7:2} means that the effect of SNP 7 is twice as great as
    snp 3 and 5, whose effects are equal. In total the snps account for 60% of the observed variance.
    
    """
    
    eps_var = 1 - beta_var
    u = 1.0 /sum([x*x for x in snp_ratios.values()])
    snp_betas = dict([(key,snp_ratios[key]*u) for key in snp_ratios.keys()])
    beta = snp_betas.values()
    snps = snp_betas.keys()
    eps_vector = np.array(np.random.normal(0,eps_var,X.shape[0]))
    return np.add(np.dot(X[:,snps], beta), eps_vector)
    
# examples
# y = simulate_traits(X,eps=0.5,snp_group={3: 5, 9: 3})

def build_linear_models(X,y):
    """
    Build univariate linear models for each SNP column in X against the trait y.
    """
    return [stats.linregress(X[:,i],y) for i in range(X.shape[1])]

# example
# models1 = [x for x in build_linear_models(X,y)]

def calc_effect_sizes(models):
    """
    Calculate the effect sizes = beta / se(beta) of individual SNPs towards the traits.
    Takes in a list of linear regression models.
    """
    return [x.slope / x.stderr for x in models]

# example
# z1 = [x.slope / x.stderr for x in models1]



array([-0.43153459,  0.49052715, -0.0227256 ,  0.22849003,  0.57955308,
       -0.25366741,  0.67519273,  1.23573824,  0.65796377,  0.04168645,
        0.16002426,  0.31421654,  0.57864023,  0.53980968, -0.06382319,
        0.73990191,  0.20510129,  0.60008869, -0.64440121, -0.13445157,
        0.9668877 ,  0.04398962, -0.45669911, -0.01619706,  0.31258278,
       -0.13836229,  0.03597163, -0.50855612, -0.13179047,  0.36871693,
        0.76215456,  0.4110753 , -0.07871894, -0.97090611, -0.02204738,
        0.28411843,  0.45376407,  0.23282093,  0.69939039,  0.04709341,
        1.37494375,  0.1820865 , -0.01096379, -0.03834385, -0.14986318,
       -0.52929852, -0.19012432, -0.04211845, -0.29511066, -0.43665341,
       -0.17941963,  0.25640165, -0.12177037, -0.44700969,  0.90434867,
        0.23900916,  0.26015724,  0.60927293,  0.31489041,  0.21704849,
       -0.05008857,  0.20852738,  0.39062797,  0.14771659,  0.39508527,
        0.42463729, -0.3491884 ,  0.62010677, -0.07521766,  0.53

In [19]:
betas

{3: 0.16666666666666666, 5: 0.16666666666666666, 10: 0.3333333333333333}

0.16666666666666666

In [4]:
X = simulate_genotype(n=100,m=30,geno_dist=[0.85,0.1,0.05])

In [6]:
simulate_traits(X,{3: 0.9, 5:0.4, 8:0.5},eps=0.5)

100

### Example

In [67]:
snp_groups = [{1: 5}, {1: 5, 3: 6}, {1: 5, 3: 6, 15:3}, {1: 5, 3: 6, 15:3, 25:1}]

for g in snp_groups:
    n = 10000

    ### simulate genotypes
    X = simulate_genotype(n=10000,m=30,geno_dist=[0.85,0.1,0.05])
    ### scale columns
    X = preprocessing.scale(X)

    ### calculate LD matrix
    LD_matrix = np.corrcoef(X,rowvar=0)

    ### simulate traits
    y = simulate_traits(X,eps=0.5,snp_group=g)
    ### scale traits
    y = preprocessing.scale(y)

    t_statistics = build_linear_models(X,y)

    beta = [x.slope for x in t_statistics]
    se_beta = [x.stderr for x in t_statistics]

    ###calcuate z

    z =  np.divide(beta, se_beta)

    simulated_effectsize_data = ([x*np.sqrt(n) for x in beta], LD_matrix, n)

    gene_set_BFs = calc_variant_set_BFs(simulated_effectsize_data,k=5,v=0.01)

    gene_set_posteriors = calc_posterior(gene_set_BFs)
    print g, gene_set_posteriors[0:5]

{1: 5} [((1,), 0.9050352515170708), ((1, 21), 0.003189029568249363), ((1, 25), 0.00318428426718765), ((1, 19), 0.003161674088441622), ((1, 4), 0.0031344545531106003)]
{1: 5, 3: 6} [((1, 3), 0.9083736228868016), ((1, 3, 13), 0.0031873943086928548), ((1, 3, 10), 0.0031412730321097606), ((1, 3, 26), 0.003138966388610109), ((1, 3, 17), 0.0031380659516668206)]
{1: 5, 3: 6, 15: 3} [((1, 3, 15), 0.9150552356086875), ((1, 3, 12, 15), 0.0031637810581904495), ((1, 3, 4, 15), 0.0031548310641800084), ((1, 3, 15, 19), 0.003154222576982418), ((1, 3, 15, 27), 0.0031541062648322094)]
{1: 5, 3: 6, 25: 1, 15: 3} [((1, 3, 15, 25), 1.0), ((1, 3, 15), 3.0459784689226484e-28), ((1, 3, 9, 15), 1.0914551735029302e-30), ((1, 3, 13, 15), 1.083211641042767e-30), ((1, 3, 15, 21), 1.0756922570942834e-30)]
